In [53]:
import numpy as np
import pandas as pd
import xarray as xr
import torch


FOLDER = 'augment_10_5'
BANDS = ['red', 'green', 'blue', 'rededge1', 'rededge2', 'rededge3', 'nir', 'swir']
VI = ['ndvi', 'savi', 'evi', 'rep','osavi','rdvi','mtvi1','lswi']
M_COLUMNS = ['tempmax', 'tempmin', 'temp', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover', 'windspeed', 'winddir', 
             'sealevelpressure', 'cloudcover', 'solarradiation', 'solarenergy', 'uvindex', 'moonphase', 'solarexposure']

S_COLUMNS = ['ndvi', 'savi', 'evi', 'rep', 'osavi', 'rdvi', 'mtvi1', 'lswi']
G_COLUMNS = ['Field size (ha)', 'Rice Crop Intensity(D=Double, T=Triple)']
TARGET = 'Rice Yield (kg/ha)'


xdf = xr.open_dataset(f'../../data/processed/{FOLDER}/train_processed.nc')
xdf

<xarray.Dataset>
Dimensions:                                  (ts_obs: 557, state_dev: 24,
                                              ts_aug: 10, datetime: 275, name: 3)
Coordinates:
    ts_id                                    (ts_obs, ts_aug) int32 ...
  * ts_obs                                   (ts_obs) int32 0 1 2 ... 555 556
  * datetime                                 (datetime) datetime64[ns] 2021-1...
  * state_dev                                (state_dev) int32 23 22 21 ... 1 0
  * ts_aug                                   (ts_aug) int32 0 1 2 3 4 5 6 7 8 9
  * name                                     (name) object 'Chau_Phu' ... 'Th...
Data variables: (12/30)
    time                                     (ts_obs, state_dev, ts_aug) datetime64[ns] ...
    ndvi                                     (ts_obs, ts_aug, state_dev) float32 ...
    savi                                     (ts_obs, ts_aug, state_dev) float32 ...
    evi                                      (ts_obs, ts_aug, state_dev) float32 ...
    rep                                      (ts_obs, ts_aug, state_dev) float32 ...
    osavi                                    (ts_obs, ts_aug, state_dev) float32 ...
    ...                                       ...
    cloudcover                               (datetime, name) float64 ...
    solarradiation                           (datetime, name) float64 ...
    solarenergy                              (datetime, name) float64 ...
    uvindex                                  (datetime, name) float64 ...
    moonphase                                (datetime, name) float64 ...
    solarexposure                            (datetime, name) float64 ...

In [54]:
xdf_id = xdf.where(xdf.ts_id == 0, drop=True)
xdf_id

<xarray.Dataset>
Dimensions:                                  (ts_obs: 1, state_dev: 24,
                                              ts_aug: 1, datetime: 275, name: 3)
Coordinates:
    ts_id                                    (ts_obs, ts_aug) int32 0
  * ts_obs                                   (ts_obs) int32 0
  * datetime                                 (datetime) datetime64[ns] 2021-1...
  * state_dev                                (state_dev) int32 23 22 21 ... 1 0
  * ts_aug                                   (ts_aug) int32 0
  * name                                     (name) object 'Chau_Phu' ... 'Th...
Data variables: (12/30)
    time                                     (ts_obs, state_dev, ts_aug) datetime64[ns] ...
    ndvi                                     (ts_obs, ts_aug, state_dev) float32 ...
    savi                                     (ts_obs, ts_aug, state_dev) float32 ...
    evi                                      (ts_obs, ts_aug, state_dev) float32 ...
    rep                                      (ts_obs, ts_aug, state_dev) float32 ...
    osavi                                    (ts_obs, ts_aug, state_dev) float32 ...
    ...                                       ...
    cloudcover                               (datetime, name, ts_obs, ts_aug) float64 ...
    solarradiation                           (datetime, name, ts_obs, ts_aug) float64 ...
    solarenergy                              (datetime, name, ts_obs, ts_aug) float64 ...
    uvindex                                  (datetime, name, ts_obs, ts_aug) float64 ...
    moonphase                                (datetime, name, ts_obs, ts_aug) float64 ...
    solarexposure                            (datetime, name, ts_obs, ts_aug) float64 ...

In [55]:
g_arr = xdf_id[G_COLUMNS].to_array().values
g_arr = g_arr.reshape(-1).astype(np.double)
g_input = torch.tensor(g_arr)
g_input

tensor([0.5888, 0.4302], dtype=torch.float64)

In [56]:
s_times = 24

s_arr = xdf_id[S_COLUMNS].to_array().values
s_arr = s_arr.reshape((len(S_COLUMNS), s_times)).T.astype(np.double)
s_input = torch.tensor(s_arr)
s_input

tensor([[-8.4131e-01, -8.4127e-01, -6.5250e-02, -9.9196e-03, -8.4130e-01,
         -7.0546e-01, -6.5202e-01, -1.0288e+00],
        [-6.0499e-01, -6.0495e-01, -5.0042e-02, -5.8247e-03, -6.0498e-01,
         -4.7210e-01, -4.2958e-01, -8.6372e-01],
        [-1.4561e-01, -1.4555e-01, -3.0782e-02,  2.7056e-04, -1.4559e-01,
          7.4499e-02,  1.8936e-01, -2.6327e-01],
        [ 3.3231e-01,  3.3239e-01, -3.4073e-02,  7.3140e-03,  3.3234e-01,
          6.4297e-01,  8.3241e-01,  4.4191e-01],
        [ 5.8956e-01,  5.8963e-01,  6.3289e-03,  9.3774e-03,  5.8958e-01,
          8.9167e-01,  1.0687e+00,  6.5434e-01],
        [ 6.6636e-01,  6.6643e-01,  3.6597e-02,  1.0444e-02,  6.6639e-01,
          9.3429e-01,  1.0839e+00,  7.0107e-01],
        [ 7.1727e-01,  7.1732e-01,  4.4787e-02,  8.0222e-03,  7.1729e-01,
          8.9689e-01,  9.7024e-01,  6.4166e-01],
        [ 6.5696e-01,  6.5700e-01,  1.0472e-02,  7.4623e-03,  6.5697e-01,
          7.4062e-01,  7.3623e-01,  4.8656e-01],
        [ 4.1040

In [57]:
m_times = 120

all_dates = pd.date_range(xdf_id['time'].min().values, xdf_id['time'].max().values, freq='D')
all_dates = all_dates[-m_times:]
g_arr = xdf_id.sel(datetime=all_dates, name=xdf_id['District'])[M_COLUMNS].to_array().values
g_arr = g_arr.reshape((len(M_COLUMNS), m_times)).T.astype(np.double)
m_input = torch.tensor(g_arr)
m_input

tensor([[ 1.0273,  0.2110,  1.0897,  ...,  0.5835, -0.0510, -0.1210],
        [ 1.6050,  0.2110,  0.8217,  ..., -0.5993,  0.0883, -0.0998],
        [ 1.6050,  0.2110,  1.1790,  ...,  0.5835,  0.1927, -0.0773],
        ...,
        [-2.1498, -0.6669, -1.7690,  ..., -1.7821, -0.0859,  1.2314],
        [-2.8430, -0.6669, -1.9477,  ..., -2.9649,  0.0186,  1.2223],
        [-1.1100, -0.6669, -0.8757,  ...,  0.5835,  0.0883,  1.2126]],
       dtype=torch.float64)

In [58]:
test = False

if test:
    target = xdf_id['Predicted Rice Yield (kg/ha)'].values
else:
    target = xdf_id['Rice Yield (kg/ha)'].values

target = target.reshape(-1)
target

array([0.10714286])

In [59]:
xdf_id['ts_obs'].values

array([0], dtype=int32)